<a href="https://colab.research.google.com/github/newmantic/interior_point/blob/main/interior_point.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np

def interior_point_method(c, A, b, x0=None, tol=1e-8, max_iter=100, reg_param=1e-8):
    """
    Improved Interior-Point Method for linear programming with regularization.

    :param c: Cost vector (1D numpy array).
    :param A: Constraint matrix (2D numpy array).
    :param b: Right-hand side vector (1D numpy array).
    :param x0: Initial guess for the solution (1D numpy array).
    :param tol: Tolerance for stopping criteria.
    :param max_iter: Maximum number of iterations.
    :param reg_param: Regularization parameter to handle singular matrices.
    :return: Optimal solution vector x.
    """
    m, n = A.shape

    # Initialize variables
    if x0 is None:
        x = np.ones(n)
    else:
        x = x0

    s = np.ones(m)
    z = np.ones(m)
    mu = 1.0

    for _ in range(max_iter):
        # Form the residuals
        r_dual = np.dot(A.T, z) - c
        r_cent = np.diag(s) @ np.diag(z) @ np.ones(m) - mu * np.ones(m)
        r_pri = np.dot(A, x) - b

        # Check stopping criteria
        if np.linalg.norm(r_dual) < tol and np.linalg.norm(r_pri) < tol and np.linalg.norm(r_cent) < tol:
            break

        # Form the KKT matrix with regularization
        KKT_matrix = np.block([
            [reg_param * np.eye(n), A.T, np.zeros((n, m))],
            [A, reg_param * np.eye(m), np.eye(m)],
            [np.zeros((m, n)), np.diag(s), np.diag(z)]
        ])

        # Form the RHS of the KKT system
        KKT_rhs = -np.hstack([r_dual, r_pri, r_cent])

        try:
            # Solve the KKT system
            delta = np.linalg.solve(KKT_matrix, KKT_rhs)
        except np.linalg.LinAlgError:
            print("Singular matrix detected. Try adjusting the regularization parameter.")
            return None

        delta_x, delta_z, delta_s = np.split(delta, [n, n + m])

        # Line search to ensure feasibility
        alpha = 1.0
        while np.any(s + alpha * delta_s <= 0) or np.any(z + alpha * delta_z <= 0):
            alpha *= 0.5

        # Update variables
        x += alpha * delta_x
        z += alpha * delta_z
        s += alpha * delta_s

        # Update mu
        mu = np.dot(s.T, z) / m

    return x


In [5]:
'''
Minimize:    2x1 + 3x2
Subject to:  x1 + 2x2 <= 2
             2x1 + x2 <= 2
             x1 >= 0, x2 >= 0
'''

def test_case_1():
    c = np.array([2, 3])
    A = np.array([[1, 2], [2, 1]])
    b = np.array([2, 2])

    x_opt = interior_point_method(c, A, b)
    print(f"Optimal solution: {x_opt}")
    # Expected: The optimal solution should be close to [0, 1]

test_case_1()

Optimal solution: [0.66666667 0.66666667]


In [7]:
'''
Minimize:    -x1 - x2 - x3
Subject to:  x1 + x2 + x3 <= 1
             2x1 + x2 <= 1
             x1 >= 0, x2 >= 0, x3 >= 0

'''


def test_case_2():
    c = np.array([-1, -1, -1])
    A = np.array([[1, 1, 1], [2, 1, 0]])
    b = np.array([1, 1])

    x_opt = interior_point_method(c, A, b)
    print(f"Optimal solution: {x_opt}")
    # Expected: The optimal solution should be close to [0.5, 0.5, 0]

test_case_2()

Optimal solution: [-8.56518702e+09 -8.59827803e+09 -8.63136905e+09]


In [8]:
'''
Minimize:    x1 + x2
Subject to:  x1 + x2 <= -1
             x1 >= 0, x2 >= 0
'''
def test_case_3():
    c = np.array([1, 1])
    A = np.array([[1, 1]])
    b = np.array([-1])

    try:
        x_opt = interior_point_method(c, A, b)
        print(f"Optimal solution: {x_opt}")
    except np.linalg.LinAlgError:
        print("Problem is infeasible.")

test_case_3()






Optimal solution: [-0.5 -0.5]
